In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### csv 파일에 tile 과 text의 내용과 함께 해당 메일의 label의 내용이 담겨 있으며 이를 Fake 와 Real News로 구분하는 Notebook을 작성해 보고자 한다.
- DNN만 이용해서 해결하기
- RNN + DNN 이용하기  
이 두가지 모델을 모두 설계해서 어떠한 모델이 제일 학습도가 높은지 확인해 보고자 한다.
  - 그렇게 하기 위해서는 다양한 모델에 적용하기 쉽도록 dataset을 만들어 주는 것이 중요하다.


In [2]:
news = pd.read_csv('/content/drive/My Drive/news.csv')

In [3]:
news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
len(news)

6335

In [5]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
groupedby_label = news.groupby('label')

In [7]:
groupedby_label.count()

,Unnamed: 0,title,text
label,,,
FAKE,3164,3164,3164
REAL,3171,3171,3171


In [8]:
news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [9]:
drop_news = news.drop(news.columns[0], axis = 1)

- column 명이 'Unnamed: 0'인 데이터는 불필요하기 때문에 그것을 drop한 dataset을 drop_news라고 새롭게 저장했다.

In [10]:
news_data = drop_news.values

- 훈련의 목적은 마지막에 2개의 class 중 하나로 분류를 할 수 있도록 하는 것이다.
- 먼저 csv파일에 있는 fake data와 real data를 따로 구분해서 title과 text문자열을 합해서 리스트에 각각 나누어 담는다.
- 그렇게 한 이후에 텍스트 전처리를 진행해야 한다.
  - 여기서 말하는 text preprocessing이란 정규식을 이용해서 '\nl'등과 같은 문자를 공백으로 바꾸고자 한다.
  - 사실상 TextVectorization층을 model의 위에 추가하면 알아서 공백에 맞추어서 나누어주고 단어들을 index화 해 주기 때문에 굳이 미리 이 작업을 해 줄 필요는 없다. 

In [211]:
fake, real = [],[]
for i in news_data:
  if i[-1] == 'FAKE':fake.append(i[0]+i[1])
  else:real.append(i[0]+i[1])
real = real[:len(fake)]

In [212]:
len(fake), len(real)

(3164, 3164)

In [45]:
(fake[3])

'Tehran, USA  \nI’m not an immigrant, but my grandparents are. More than 50 years ago, they arrived in New York City from Iran. I grew up mainly in central New Jersey, an American kid playing little league for the Raritan Red Sox and soccer for the Raritan Rovers. In 1985, I travelled with my family to our ancestral land. I was only eight, but old enough to understand that the Iranians had lost their liberty and freedom. I saw the abject despair of a people who, in a desperate attempt to bring about change, had ushered in nationalist tyrants led by Ayatollah Khomeini. \nWhat I witnessed during that year in Iran changed the course of my life. In 1996, at age 19, wanting to help preserve the blessings of liberty and freedom we enjoy in America, I enlisted in the U.S. Navy. Now, with the rise of Donald Trump and his nationalist alt-right movement, I’ve come to feel that the values I sought to protect are in jeopardy. \nIn Iran, theocratic fundmentalists sowed division and hatred of outsid

- 간단하게 preprocess 함수를 만들어 보자면 아래와 같다.
  - 정규화 조건을 이용했기 때문에 우선 문자가 아닌 값들은(알파벳이 아닌 값) 모두 공백으로 처리 했고, '\n'과 같은 띄어쓰기 등을 의미하는 값들 또한 모두 공백으로 바꾸어 주었다.

In [210]:
def preprocess_fake(x):
  x = tf.strings.substr(x, 0, 300)
  x = tf.strings.regex_replace(x, b"<br\\s*/?>", b" ")
  x = tf.strings.regex_replace(x, b"[^a-zA-Z']", b" ")
  x = tf.strings.split(x)
  return x.to_tensor(default_value = b"<pad>"), tf.constant([0])

def preprocess_real(x):
  x = tf.strings.substr(x, 0, 300)
  x = tf.strings.regex_replace(x, b"<br\\s*/?>", b" ")
  x = tf.strings.regex_replace(x, b"[^a-zA-Z']", b" ")
  x = tf.strings.split(x)
  return x.to_tensor(default_value = b"<pad>"), tf.constant([1])

In [213]:
from sklearn.model_selection import train_test_split
fake_train, fake_test = train_test_split(fake, test_size = 0.2)
fake_train, fake_val = train_test_split(fake_train, test_size = 0.2)
real_train, real_test = train_test_split(real, test_size = 0.2)
real_train, real_val = train_test_split(real_train, test_size = 0.2)

In [195]:
len(real_train), len(fake_test), len(fake_val)

(2024, 633, 507)

In [49]:
fake_train[0]

"Valentin Katasonov: America is in agony and Trump is the doctor\n\nNovember 12, 2016 - Fort Russ \n\nNeyromir TV (Video)  - Valentin Katasonov - Translated from Russian by Kristina Kharlova \n\n\nThe heated discussion in the media is a diversion - focused on marginal issues appealing to emotions, while much graver issues that are at stake are hidden behind the scenes, explains Valentin Katasonov, p rofessor, associate member of the Russian Academy of Economic Science and Business. \n\n\n\n\nPart 1 (00.00-14.00) \n\n\n\n\n\nV.K: Trump understands the situation. I didn't expect him to be so open about revealing all the ills. He is revealing many of the secrets. This is better for America - to face the diagnosis, than to conceal it from  the patient.  \n\nAs you know interest rates in the countries of the Golden billion are below the floor.  Last year when interest rates were slightly raised, this caused serious consequences . Christine Lagarde appealed to stop or the global economy will

DATA 1-1. DNN Layer을 위한 data 만들기
- 어차피 TextVectorization을 이용할 것이기 때문에 따로 전처리를 진행한 것은 아니고 그냥 x, y를 반한하도록 했다.

In [51]:
def preprocess_dnn_fake(x):
  return x,0
def preprocess_dnn_real(x):
  return x,1

fake_train_dnn = tf.data.Dataset.from_tensor_slices(fake_train).map(preprocess_dnn_fake)
fake_test_dnn = tf.data.Dataset.from_tensor_slices(fake_test).map(preprocess_dnn_fake)  
fake_val_dnn = tf.data.Dataset.from_tensor_slices(fake_val).map(preprocess_dnn_fake)  


real_train_dnn = tf.data.Dataset.from_tensor_slices(real_train).map(preprocess_dnn_real)
real_test_dnn = tf.data.Dataset.from_tensor_slices(real_test).map(preprocess_dnn_real)  
real_val_dnn = tf.data.Dataset.from_tensor_slices(real_val).map(preprocess_dnn_real)  


In [84]:
train_dnn = tf.data.Dataset.concatenate(fake_train_dnn, real_train_dnn).repeat().shuffle(2024).batch(32)
test_dnn = tf.data.Dataset.concatenate(fake_test_dnn, real_test_dnn).batch(32)
val_dnn = tf.data.Dataset.concatenate(fake_val_dnn, real_val_dnn).batch(32)

In [188]:
for i,j in train_dnn.take(1):
  print(j)

tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(32,), dtype=int32)


DATA 1-2. DNN + RNN Layer을 위한 data 만들기
- preprocess 함수를 이용해서 미리 데이터의 적재와 전처리를 해준다.
- 알파벳 제외 다른 것들은 공백으로 남겨 두는데, 무조건 마지막에 모든 텍스트 데이터의 길이를 동일하게 설정하기 위해서 dafault_value는 <pad>로 바꿔주는 과정을 거친다.

In [214]:
BATCH_SIZE = 32
fake_train = tf.data.Dataset.from_tensor_slices(fake_train).batch(BATCH_SIZE).map(preprocess_fake)
fake_test = tf.data.Dataset.from_tensor_slices(fake_test).batch(BATCH_SIZE).map(preprocess_fake)
fake_val = tf.data.Dataset.from_tensor_slices(fake_val).batch(BATCH_SIZE).map(preprocess_fake)

In [215]:
for x,y in fake_train.take(1):
  print(y)

tf.Tensor([0], shape=(1,), dtype=int32)


In [216]:
real_train = tf.data.Dataset.from_tensor_slices(real_train).batch(BATCH_SIZE).map(preprocess_real)
real_test = tf.data.Dataset.from_tensor_slices(real_test).batch(BATCH_SIZE).map(preprocess_real)
real_val = tf.data.Dataset.from_tensor_slices(real_val).batch(BATCH_SIZE).map(preprocess_real)

In [217]:
BUFFER_SIZE = 2024
train_dataset = tf.data.Dataset.concatenate(fake_train, real_train).shuffle(BUFFER_SIZE)
test_dataset = tf.data.Dataset.concatenate(fake_test, real_test)
val_dataset = tf.data.Dataset.concatenate(fake_val, real_val)

In [218]:
for x,y in train_dataset.take(1):
  print(y)

tf.Tensor([0], shape=(1,), dtype=int32)


### 1. RNN + DNN Layer
- 이 방법으로 하기 위해서 직접 단어 사전을 만들어서 단어를 인코딩을 했다.

In [219]:
from collections import Counter
vocab = Counter()
for x, y in train_dataset:
  for text in x:
    vocab.update(list(text.numpy()))

In [220]:
train_dataset = train_dataset.repeat()

In [221]:
vocab.most_common()[:5]

[(b'<pad>', 38896), (b'the', 7443), (b'to', 4256), (b'of', 4114), (b'a', 3535)]

**주어진 텍스트를 이용해서 단어 사전을 만드는 과정**  

1. 먼저 Counter이라는 함수를 불러서 해당 텍스트 데이터에 있는 모든 단어를 dictionaty의 형태에 넣어 각 단어가 몇번 이 나왔는지 자동으로 저장해 준다.
2. 그 단어 사전에 저장된 단어들 중 우리가 유효하게 의미를 생각할 단어의 개수를 정해주고 most_common함수를 이용해서 가장 많이 쓰인 것만 저장을 따로 해준다.
3. tensor의 형태로 바꾸어 준 뒤에 개수만큼 정수가 나열된 list를 만든다.
4. 이후 ```tf.lookup.KeyValueTensorInitializer```을 이용해서 단어와 단어의 인덱스를 match해주는 lookup 도구를 만든다.
5. 마지막으로 혹시 모를 사전에 없을 단어들을 위해서 num_oov_bucket까지 추가 해 주어 학습에 사용될 데이터에 적용될 lookup table을 만든다.

In [222]:
vocab_size = 10000
vocab_dict = [word for word, count in vocab.most_common()[:vocab_size]]

words = tf.constant(vocab_dict)
words_ind = tf.range(len(vocab_dict), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, words_ind)
num_oov_bucket = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_bucket)

def encode_word(x,y):
  return table.lookup(x), y

train_data = train_dataset.map(encode_word)
test_data = test_dataset.map(encode_word)
val_data = val_dataset.map(encode_word)

In [223]:
for i,j in train_data.take(1):print(i,j)

tf.Tensor(
[[   93    54  1748 ...     0     0     0]
 [  859    41  1635 ...     0     0     0]
 [ 3847    30  2560 ...     0     0     0]
 ...
 [10186  6272 10527 ...     0     0     0]
 [  180  1839  1039 ...     0     0     0]
 [  119   448  3413 ...     0     0     0]], shape=(32, 54), dtype=int64) tf.Tensor([1], shape=(1,), dtype=int32)


In [233]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.activations import sigmoid

embed_size = 128
model = Sequential()
model.add(Embedding(11000, embed_size, input_shape = [None]))
model.add(GRU(128, return_sequences = True))
model.add(GRU(128))
model.add(Dense(100, activation = 'selu'))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 128)         1408000   
_________________________________________________________________
gru_15 (GRU)                 (None, None, 128)         99072     
_________________________________________________________________
gru_16 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_55 (Dense)             (None, 100)               12900     
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 101       
Total params: 1,619,145
Trainable params: 1,619,145
Non-trainable params: 0
_________________________________________________________________


- 이 모델에 fitting 할 때에 처음에는 출력층의 shape와 label의 shape가 일치 하지 않았어서 문제가 많이 발생했었다. 
- 그래서 보니까 dataset에 저장된 label이 shape = ()으로 None으로 지정이 되어 있음을 알 수 있었다. 그래서 preprocessing 함수에서 y의 값을 그냥 상수가 아니라 [0]과 [1]로 반환해 주었더니 shape가 생겼기 때문에 문제 없이 모델을 학습할 수 있었다.

In [234]:
model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
history = model.fit(train_data, validation_data = val_data, epochs = 10, steps_per_epoch = 2024//BATCH_SIZE)

Epoch 1/10
63/63 [==============================] - 10s 156ms/step - loss: 0.7037 - accuracy: 0.4573 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
63/63 [==============================] - 9s 143ms/step - loss: 0.6963 - accuracy: 0.4675 - val_loss: 0.6942 - val_accuracy: 0.5079
Epoch 3/10
63/63 [==============================] - 9s 141ms/step - loss: 0.7267 - accuracy: 0.5574 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 4/10
63/63 [==============================] - 9s 143ms/step - loss: 0.7085 - accuracy: 0.5243 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 5/10
63/63 [==============================] - 9s 146ms/step - loss: 0.6878 - accuracy: 0.5308 - val_loss: 0.6853 - val_accuracy: 0.5108
Epoch 6/10
63/63 [==============================] - 9s 144ms/step - loss: 0.5808 - accuracy: 0.7282 - val_loss: 0.6304 - val_accuracy: 0.7041
Epoch 7/10
63/63 [==============================] - 9s 144ms/step - loss: 0.3564 - accuracy: 0.8674 - val_loss: 0.4815 - val_accuracy: 0.7771
Epoch

In [235]:
model.evaluate(test_data)

40/40 [==============================] - 1s 32ms/step - loss: 0.3338 - accuracy: 0.9028


[0.33382105827331543, 0.9028435945510864]

### Accuracy = 90.28%

### 2. DNN Layer Only
- tf.keras.layers.TextVectorzation 층을 이용해서 범주형 데이터를 수치형 데이터로 바꾸어 준다.
- 그리고 사용자 정의 Standardization 층을 만들어서 이렇게 인덱스된 데이터의 각 단어별 id를 만들어 주고자 한다.
  - TextVectorization을 안 사용한다면 직접 해당 텍스트 데이터에 있는 단어들을 이용해서 단어 사전을 만들어야 할 것이다.

- 반드시 모델의 loss는 binary_crossentropy여야 한다. categorical_crossentropy는 class의 개수가 더 많을 때에 사용한다.  

In [129]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vect_layer = TextVectorization(max_tokens = 11000, output_mode = 'int', output_sequence_length = 500)
vect_layer.adapt(test_dnn.map(lambda x,y:x))

In [130]:
for i,j in test_dnn.take(1):
  print(vect_layer(i))

tf.Tensor(
[[5486    1  358 ...   45 2121    3]
 [ 225 9066   54 ...   27   44    4]
 [ 164   37  922 ...   26    8    1]
 ...
 [2119 1276  168 ...    3   68 8673]
 [ 164  132 1308 ...  953 5897  416]
 [1725 4816    9 ...    0    0    0]], shape=(32, 500), dtype=int64)


In [131]:
class BagOfWords(tf.keras.layers.Layer):
  def __init__(self, n_tokens, dtype = tf.int32):
    super().__init__(dtype = tf.int32)
    self.n_tokens = n_tokens
  def call(self, inputs):
    one_hot = tf.one_hot(inputs, self.n_tokens)
    return tf.reduce_sum(one_hot, axis = 1)[:, 1:]

In [134]:
bag_of_words = BagOfWords(500)

In [135]:
for i,j in test_dnn.take(1):
  print(vect_layer(i))
  print(bag_of_words(vect_layer(i)))

tf.Tensor(
[[    2 10297     1 ...   164  1272     2]
 [    2  1675     1 ...     0     0     0]
 [   89     3   389 ...  1824    76  3177]
 ...
 [    2     1     7 ...   396    31    17]
 [  425     1   149 ...     0     0     0]
 [   35    36   664 ...     0     0     0]], shape=(32, 500), dtype=int64)
tf.Tensor(
[[54. 33. 15. ...  0.  0.  0.]
 [34. 43. 10. ...  0.  0.  0.]
 [44. 14. 29. ...  0.  0.  0.]
 ...
 [47. 29. 11. ...  0.  0.  0.]
 [41. 17. 12. ...  0.  0.  1.]
 [ 6. 10.  4. ...  0.  0.  0.]], shape=(32, 499), dtype=float32)


**아래 모델을 설계할 때에 반드시 class가 2개이기 때문에 

In [138]:
model = Sequential()
model.add(vect_layer)
model.add(bag_of_words)
model.add(Dense(100, activation = 'relu'))
model.add(Dense(300, activation = 'relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
history = model.fit(train_dnn, validation_data= val_dnn, epochs = 20, steps_per_epoch= 2024//32)

Epoch 1/20
63/63 [==============================] - 3s 44ms/step - loss: 0.5334 - accuracy: 0.7307 - val_loss: 0.3947 - val_accuracy: 0.8097
Epoch 2/20
63/63 [==============================] - 3s 40ms/step - loss: 0.3789 - accuracy: 0.8343 - val_loss: 0.4331 - val_accuracy: 0.7850
Epoch 3/20
63/63 [==============================] - 3s 41ms/step - loss: 0.3743 - accuracy: 0.8398 - val_loss: 0.3434 - val_accuracy: 0.8560
Epoch 4/20
63/63 [==============================] - 3s 41ms/step - loss: 0.3564 - accuracy: 0.8557 - val_loss: 0.3585 - val_accuracy: 0.8521
Epoch 5/20
63/63 [==============================] - 3s 41ms/step - loss: 0.3327 - accuracy: 0.8522 - val_loss: 0.3045 - val_accuracy: 0.8728
Epoch 6/20
63/63 [==============================] - 3s 41ms/step - loss: 0.2728 - accuracy: 0.8874 - val_loss: 0.3212 - val_accuracy: 0.8600
Epoch 7/20
63/63 [==============================] - 3s 43ms/step - loss: 0.3011 - accuracy: 0.8740 - val_loss: 0.3740 - val_accuracy: 0.8304
Epoch 8/20
63

In [139]:
model.evaluate(test_dnn)

40/40 [==============================] - 1s 26ms/step - loss: 0.4078 - accuracy: 0.8728


[0.40784668922424316, 0.8728278279304504]

### Accuracy = 87.28%

In [144]:
test_dnn = test_dnn.shuffle(100)

In [150]:
for i, j in test_dnn.take(20):
  pred = model.predict(i)
  if pred[0] < 1-pred[0]:print((j[0].numpy(), 0))
  else:print((j[0].numpy(), 1))

(1, 1)
(0, 1)
(0, 0)
(0, 0)
(1, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(1, 0)
(1, 1)
(1, 1)
(0, 0)
(1, 1)
(1, 0)
(0, 0)
(1, 1)
(1, 1)


### 결론
1. 텍스트 전처리를 할 수 있는 방법은 정말 많지만, 이번에는 두가지 방법을 진행해 보았다.
  - 아무래도 문자 사이의, 그리고 문맥 사이의 유사도를 찾아서 indexing해주는 Embeddding layer을 포함한 RNN layer의 정확도가 더 높았다.
  - 진행한 두가지 방법은 
    1. 먼저 불필요한 문자나 여백 등은 제거하고 vocab lookup set를 직접 만들고 이를 적용해서 문장들을 인덱싱 해준 후에 Embedding Layer과 GRU순환 신경망을 이용해서 단어사이의 유사도를 탐색해 학습을 진행한다.
    2. 전처리는 따로 해 주지 않고 label과 mapping만 해 준 이후에 TextVectorization과 BoxOfWords를 이용해서 학습을 해 준다. 이때는 심층 신경망만 이용을 했다.
2. 아무래도 2개의 class로 나누는 것이다 보니까 loss를 반드시 'binary_crossentropy'로 설정 해 주어야만 했다.    